# Assignment 0: UserProxyAgent Integration in Society of Mind Teams

**Author**: Student Name  
**Due Date**: August 9th, 2025  
**Objective**: Demonstrate human-in-the-loop implementation using UserProxyAgent in Microsoft AutoGen's Society of Mind (SoM) framework

## Architecture Overview

This implementation demonstrates both **Inner Team Integration** and **Outer Team Integration** with strategic UserProxyAgent placement for human oversight at critical decision points.

## Flow Diagram (Simplified Architecture)

```
┌─────────────────────────────────────────────────────────────┐
│                   OUTER TEAM COORDINATOR                    │
│  ┌─────────────────┐    ┌─────────────────────────────────┐ │
│  │ ContentTeam_SoM │    │     QualityTeam_SoM             │ │
│  │ (Society of     │    │     (Society of                 │ │
│  │  Mind Agent)    │    │      Mind Agent)                │ │
│  └─────────────────┘    └─────────────────────────────────┘ │
│           │                              │                  │
│           ▼                              ▼                  │
│  ┌─────────────────────┐      ┌─────────────────────────┐   │
│  │   INNER TEAM A      │      │    INNER TEAM B         │   │
│  │  (Content Creation) │      │  (Quality Assurance)    │   │
│  │  ┌─────────┐        │      │  ┌─────────┐             │   │
│  │  │ Writer  │        │      │  │Reviewer │             │   │
│  │  └─────────┘        │      │  └─────────┘             │   │
│  │  ┌─────────┐        │      │  ┌─────────┐             │   │
│  │  │ Editor  │        │      │  │ Human   │             │   │
│  │  └─────────┘        │      │  │Overseer │             │   │
│  │  ┌─────────┐        │      │  └─────────┘             │   │
│  │  │ Human   │        │      │                          │   │
│  │  │Overseer │        │      │                          │   │
│  │  └─────────┘        │      │                          │   │
│  └─────────────────────┘      └─────────────────────────┘   │
│                              │                              │
│                              ▼                              │
│                    ┌─────────────────┐                      │
│                    │ Human Project   │                      │
│                    │ Overseer        │                      │
│                    │ (Final Approval)│                      │
│                    └─────────────────┘                      │
└─────────────────────────────────────────────────────────────┘

Simplified Human Intervention Points:
• Inner Team A: Content approval (Writer → Editor → Human)
• Inner Team B: Quality validation (Reviewer → Human)
• Outer Team: Final project approval (Human oversight)
```

In [2]:
import os
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent, SocietyOfMindAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.ui import Console
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')

# Initialize the model client
model_client = OpenAIChatCompletionClient(model='gemini-2.5-flash', api_key=api_key)

## Part A: Inner Team Integration 
### Content Creation Inner Team
This inner team consists of specialized agents for content creation with human oversight at critical decision points.

In [3]:
async def create_content_creation_team():
    """
    Creates Inner Team A: Content Creation Team (Simplified)
    - Writer Agent: Creates initial content
    - Editor Agent: Reviews and refines content
    - UserProxy Agent: Human intervention for approval/feedback
    """
    
    # Writer Agent - Specialized for content creation
    writer_agent = AssistantAgent(
        name='Writer',
        description='Expert content writer',
        model_client=model_client,
        system_message='''
        You are a content writer. Create engaging, well-structured content based on requirements.
        Keep responses concise. Always end with "READY_FOR_REVIEW" when presenting content.
        '''
    )
    
    # Editor Agent - Specialized for content refinement
    editor_agent = AssistantAgent(
        name='Editor',
        description='Content editor for quality assurance',
        model_client=model_client,
        system_message='''
        You are a content editor. Review content for clarity and quality.
        Provide constructive feedback and suggestions.
        End with "CONTENT_APPROVED" if satisfactory, or "NEEDS_REVISION" with feedback.
        '''
    )
    
    # UserProxy Agent for human intervention
    def human_input_function(prompt):
        print("\n" + "="*50)
        print(" HUMAN INPUT - Content Team")
        print("="*50)
        print("Options:")
        print("1. Type 'APPROVE' to approve")
        print("2. Provide feedback for improvements")
        print("-"*50)
        return input("Your decision: ")
    
    user_proxy_inner_a = UserProxyAgent(
        name='Human_ContentOverseer',
        description='Human oversight for content decisions',
        input_func=human_input_function
    )
    
    # Termination condition
    inner_termination_a = TextMentionTermination("APPROVE")
    
    # Create inner team - Simple 2 agents + human
    inner_team_a = RoundRobinGroupChat(
        participants=[writer_agent, editor_agent, user_proxy_inner_a],
        termination_condition=inner_termination_a,
        max_turns=4
    )
    
    return inner_team_a

### Quality Assurance Inner Team
This inner team focuses on quality assurance, validation, and final polishing with human oversight.

In [4]:
async def create_quality_assurance_team():
    """
    Creates Inner Team B: Quality Assurance Team (Simplified)
    - Reviewer Agent: Quality review
    - UserProxy Agent: Human oversight for quality decisions
    """
    
    # Reviewer Agent - Quality assessment
    reviewer_agent = AssistantAgent(
        name='Reviewer',
        description='Quality reviewer for content assessment',
        model_client=model_client,
        system_message='''
        You are a quality reviewer. Check content for accuracy, completeness, and standards.
        Provide clear feedback. End with "QUALITY_APPROVED" if acceptable, or "NEEDS_IMPROVEMENT" with issues.
        '''
    )
    
    # UserProxy Agent for quality decisions
    def quality_input_function(prompt):
        print("\n" + "="*50)
        print(" HUMAN INPUT - Quality Team")
        print("="*50)
        print("Options:")
        print("1. Type 'QUALITY_APPROVED' to approve")
        print("2. Provide quality feedback")
        print("-"*50)
        return input("Your quality decision: ")
    
    user_proxy_inner_b = UserProxyAgent(
        name='Human_QualityOverseer',
        description='Human oversight for quality decisions',
        input_func=quality_input_function
    )
    
    # Termination condition
    inner_termination_b = TextMentionTermination("QUALITY_APPROVED")
    
    # Create inner team - Simple 1 agent + human
    inner_team_b = RoundRobinGroupChat(
        participants=[reviewer_agent, user_proxy_inner_b],
        termination_condition=inner_termination_b,
        max_turns=3
    )
    
    return inner_team_b

## Part B: Outer Team Integration

### Society of Mind Agents and Team Coordination
This section creates SocietyOfMindAgent wrappers for inner teams and establishes outer team coordination.

In [5]:
async def create_outer_team_coordination():
    """
    Creates the outer team structure (Simplified):
    - Society of Mind agents wrapping inner teams
    - UserProxy for high-level oversight
    """
    
    # Create inner teams
    inner_team_a = await create_content_creation_team()
    inner_team_b = await create_quality_assurance_team()
    
    # Create Society of Mind agent for Content Creation team
    som_content_agent = SocietyOfMindAgent(
        name="ContentTeam_SoM",
        team=inner_team_a,
        model_client=model_client,
        response_prompt='''
        Summarize the content creation process and provide the final content.
        Keep the response clear and concise.
        '''
    )
    
    # Create Society of Mind agent for Quality Assurance team
    som_quality_agent = SocietyOfMindAgent(
        name="QualityTeam_SoM",
        team=inner_team_b,
        model_client=model_client,
        response_prompt='''
        Summarize the quality review process and provide final approval status.
        Keep the response clear and concise.
        '''
    )
    
    # UserProxy Agent for outer team oversight
    def outer_team_input_function(prompt):
        print("\n" + "="*60)
        print(" HUMAN INPUT - Outer Team Coordination")
        print("="*60)
        print("Options:")
        print("1. Type 'FINAL_APPROVAL or 1' for final approval")
        print("2. Type 'REJECT_OUTPUT' to reject and rework")
        print("3. Provide coordination feedback")
        print("-"*60)
        return input("Your coordination decision: ")
    
    outer_user_proxy = UserProxyAgent(
        name='Human_ProjectOverseer',
        description='Human oversight for final decisions',
        input_func=outer_team_input_function
    )
    
    # Outer team termination condition
    outer_termination = TextMentionTermination("FINAL_APPROVAL")


    # Create outer team - Simple structure
    outer_team = RoundRobinGroupChat(
        participants=[som_content_agent, som_quality_agent, outer_user_proxy],
        termination_condition=outer_termination,
        max_turns=4
    )
    
    return outer_team

## Demonstration: 
This section demonstrates the complete integration of both inner and outer teams with UserProxyAgent oversight at multiple levels.

In [6]:
async def demonstrate_complete_integration():
    """
    Demonstrates the Society of Mind implementation with:
    - Inner teams with human-in-the-loop
    - Outer team coordination with strategic human oversight
    """
    
    # Create the outer team structure
    outer_team = await create_outer_team_coordination()
    
    # Task
    task = """
    Create a 50 word blog post about "AI in Healthcare" that:
    1. Includes current applications
    2. Mentions key challenges
    3. Concludes with future outlook
    
    Keep it accessible and informative.
    """
    
    print("\n Starting collaboration...\n")
    
    # Run the outer team with the task
    stream = outer_team.run_stream(task=task)
    await Console(stream)
    
    print("\n" + "="*60)
    print(" DEMONSTRATION COMPLETED")
    print("="*60)

## Execution Section
**Complete Integration**: Full demonstration with all features

**Note**: Make sure to set your Google API key in the environment variables or update the `api_key` variable above before running.

In [7]:
await demonstrate_complete_integration()


 Starting collaboration...

---------- TextMessage (user) ----------



    Create a 50 word blog post about "AI in Healthcare" that:
    1. Includes current applications
    2. Mentions key challenges
    3. Concludes with future outlook

    Keep it accessible and informative.
    
---------- TextMessage (Writer) ----------
AI is transforming healthcare, accelerating drug discovery, enhancing diagnostics, and personalizing treatment plans. Current applications include radiology analysis and predictive analytics. Key challenges involve data privacy, ethical considerations, and regulatory hurdles. Despite this, AI's future promises unprecedented efficiency, precision medicine, and improved patient outcomes, revolutionizing global health.

READY_FOR_REVIEW

==================================================---------- TextMessage (Editor) ----------

 HUMAN INPUT - Content Team
Options:
1. Type 'APPROVE' to approve
2. Provide feedback for improvements
--------------------------------------------------
The blog post effectively covers the required points: cu

## Code Documentation (Architecture)

### Agent Roles and Responsibilities

#### Inner Team A - Content Creation:
- **Writer**: Creates initial content based on requirements
- **Editor**: Reviews and refines content for quality
- **Human_ContentOverseer**: Provides human oversight for content decisions

#### Inner Team B - Quality Assurance:
- **Reviewer**: Conducts quality assessment of content
- **Human_QualityOverseer**: Provides human validation for quality standards

#### Outer Team - Coordination:
- **ContentTeam_SoM**: Society of Mind agent wrapping Inner Team A
- **QualityTeam_SoM**: Society of Mind agent wrapping Inner Team B
- **Human_ProjectOverseer**: Strategic human oversight for final decisions

### Human Intervention Points

1. **Inner Team Level**:
   - Content Team: Approve/reject content, provide feedback
   - Quality Team: Approve quality standards, request improvements

2. **Outer Team Level**:
   - Final project approval or rejection
   - Strategic coordination decisions

### Key Features Implemented

- **Simplified multi-agent inner teams** (2-3 agents each)  
-  **UserProxyAgent integration** at critical decision points  
-  **Clean human feedback loops** with clear approval/rejection  
-  **Society of Mind coordination** managing inner teams  
-  **Strategic human oversight** at outer level  
-  **Reduced complexity** while maintaining core functionality  
-  **Clear documentation** of simplified roles and processes  

This simplified implementation demonstrates a complete Society of Mind architecture with human-in-the-loop capabilities at both inner and outer team levels, fulfilling all requirements of Assignment 0 with much cleaner, more manageable complexity.